In [26]:
!pip install scikit-image
!pip install torchmetrics


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import torch.utils.data as data


In [28]:
from torch.utils.data import Dataset
from PIL import Image
import os

class DenoisingDataset(Dataset):
    def __init__(self, noisy_dir, clean_dir, transform=None):
        self.noisy_dir = noisy_dir
        self.clean_dir = clean_dir
        self.noisy_images = os.listdir(noisy_dir)
        self.clean_images = os.listdir(clean_dir)
        self.transform = transform

        # Ensure the length of noisy and clean images matches
        assert len(self.noisy_images) == len(self.clean_images), "Mismatch between noisy and clean images"

    def __len__(self):
        return len(self.noisy_images)

    def __getitem__(self, idx):
        # Get the file names
        noisy_image_name = self.noisy_images[idx]
        clean_image_name = self.clean_images[idx]

        # Load noisy and clean images
        noisy_image_path = os.path.join(self.noisy_dir, noisy_image_name)
        clean_image_path = os.path.join(self.clean_dir, clean_image_name)

        noisy_image = Image.open(noisy_image_path).convert("RGB")
        clean_image = Image.open(clean_image_path).convert("RGB")

        if self.transform:
            noisy_image = self.transform(noisy_image)
            clean_image = self.transform(clean_image)

        return noisy_image, clean_image


In [29]:
# Define transformations
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize((0.5,), (0.5,)) , # Normalize pixel values
    transforms.Resize((256, 256)),  # Resize images to 256x256
    transforms.RandomHorizontalFlip()  
])
test_transform = transforms.Compose([
    transforms.Resize((256, 256)),  
    transforms.ToTensor(),  
    transforms.Normalize((0.5,), (0.5,))  
])


# Paths to noisy and clean datasets
noisy_train_dir = 'denoising-dirty-documents/train/train'
clean_train_dir = 'denoising-dirty-documents/train_cleaned/train_cleaned'
test_dir = 'denoising-dirty-documents/test'

# Create the dataset
train_data = DenoisingDataset(noisy_dir=noisy_train_dir, clean_dir=clean_train_dir, transform=transform)
train_size = int(0.85 * len(train_data))
val_size = len(train_data) - train_size

train_dataset, val_dataset = data.random_split(train_data, [train_size, val_size])

# Create the DataLoader
train_loader = data.DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = data.DataLoader(val_dataset, batch_size=32, shuffle=False)  
test_data = datasets.ImageFolder(test_dir, transform=test_transform)
test_loader = data.DataLoader(test_data, batch_size=32, shuffle=True)
print("Number of test samples:", len(test_data))
print("Number of validation samples:", len(val_dataset))

print("Number of training samples:", len(train_dataset))

Number of test samples: 72
Number of validation samples: 22
Number of training samples: 122


In [2]:
class DenoisingAutoencoder(nn.Module):
    def __init__(self):
        super(DenoisingAutoencoder, self).__init__()
        # Encoder
        self.encoder1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=2, padding=1),  
            nn.BatchNorm2d(64),
            nn.ReLU()
        )
        self.encoder2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),  # [batch, 128, 64, 64]
            nn.BatchNorm2d(128),
            nn.ReLU()
        )
        self.encoder3 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1),  # [batch, 256, 32, 32]
            nn.BatchNorm2d(256),
            nn.ReLU()
        )
        
        # Decoder with skip connections
        self.decoder1 = nn.Sequential(
            nn.ConvTranspose2d(256, 128, kernel_size=3, stride=2, padding=1, output_padding=1),  # [batch, 128, 64, 64]
            nn.BatchNorm2d(128),
            nn.ReLU()
        )
        self.decoder2 = nn.Sequential(
            nn.ConvTranspose2d(256, 64, kernel_size=3, stride=2, padding=1, output_padding=1),  # [batch, 64, 128, 128]
            nn.BatchNorm2d(64),
            nn.ReLU()
        )
        self.decoder3 = nn.Sequential(
            nn.ConvTranspose2d(128, 3, kernel_size=3, stride=2, padding=1, output_padding=1),  # [batch, 3, 256, 256]
            nn.Sigmoid()  # We use Sigmoid to ensure the output is between 0 and 1
        )
    
    def forward(self, x):
        # Encode
        enc1 = self.encoder1(x)
        enc2 = self.encoder2(enc1)
        enc3 = self.encoder3(enc2)
        
        # Decode with skip connections
        dec1 = self.decoder1(enc3)
        dec1 = torch.cat([dec1, enc2], dim=1)  # Skip connection from encoder2
        dec2 = self.decoder2(dec1)
        dec2 = torch.cat([dec2, enc1], dim=1)  # Skip connection from encoder1
        dec3 = self.decoder3(dec2)
        
        return dec3




In [ ]:
from torchvision import models
from torchsummary import summary
model = DenoisingAutoencoder().cuda()  # Move the model to GPU
summary(model, (3, 256, 256))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 128, 128]           1,792
       BatchNorm2d-2         [-1, 64, 128, 128]             128
              ReLU-3         [-1, 64, 128, 128]               0
            Conv2d-4          [-1, 128, 64, 64]          73,856
       BatchNorm2d-5          [-1, 128, 64, 64]             256
              ReLU-6          [-1, 128, 64, 64]               0
            Conv2d-7          [-1, 256, 32, 32]         295,168
       BatchNorm2d-8          [-1, 256, 32, 32]             512
              ReLU-9          [-1, 256, 32, 32]               0
  ConvTranspose2d-10          [-1, 128, 64, 64]         295,040
      BatchNorm2d-11          [-1, 128, 64, 64]             256
             ReLU-12          [-1, 128, 64, 64]               0
  ConvTranspose2d-13         [-1, 64, 128, 128]         147,520
      BatchNorm2d-14         [-1, 64, 1

In [31]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [32]:
import time
import torch
from tqdm import tqdm
import torch.nn.functional as F
import numpy as np

from torchmetrics import StructuralSimilarityIndexMeasure

def calculate_ssim_batch(clean_images: torch.Tensor, outputs: torch.Tensor) -> torch.Tensor:
    """
    Calculate SSIM for a batch of images.

    Args:
        clean_images (torch.Tensor): A tensor of shape (N, C, H, W) for ground truth images.
        outputs (torch.Tensor): A tensor of shape (N, C, H, W) for generated or predicted images.
        device (torch.device): The device (CPU or GPU) to run the calculation on.

    Returns:
        torch.Tensor: A tensor containing SSIM values for each image in the batch.
    """
    # Initialize the SSIM metric on the specified device
    ssim = StructuralSimilarityIndexMeasure(data_range=1.0).to(device)

    # Ensure both input tensors are of the same shape
    if clean_images.shape != outputs.shape:
        raise ValueError("The shape of clean_images and outputs must match.")

    # Move input tensors to the same device
    clean_images = clean_images.to(device)
    outputs = outputs.to(device)

    # Calculate SSIM for the entire batch
    ssim_values = ssim(clean_images, outputs)

    return ssim_values

In [33]:
def train_and_evaluate_autoencoder(model, train_loader, val_loader, optimizer, scheduler, num_epochs):
    training_data = {
        'train_loss': [],
        'val_loss': [],
        'mse': [],
        'ssim': []
    }

    start_time = time.time()

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        total_train_loss = 0
        
        train_loader_iter = tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{num_epochs}")

        for noisy_images, clean_images in train_loader_iter:
            noisy_images, clean_images = noisy_images.to(device), clean_images.to(device)

            optimizer.zero_grad()
            outputs = model(noisy_images)
            loss = F.mse_loss(outputs, clean_images)
            loss.backward()
            optimizer.step()

            total_train_loss += loss.item() * noisy_images.size(0)

        scheduler.step()
        avg_train_loss = total_train_loss / len(train_loader.dataset)
        
        # Store training loss
        training_data['train_loss'].append(avg_train_loss)

        # Validation phase
        model.eval()
        total_val_loss = 0
        mse_list = []
        ssim_list = []

        val_loader_iter = tqdm(val_loader, desc=f"Validating Epoch {epoch+1}/{num_epochs}")

        with torch.no_grad():
            for noisy_images, clean_images in val_loader_iter:
                noisy_images, clean_images = noisy_images.to(device), clean_images.to(device)

                outputs = model(noisy_images)
                loss = F.mse_loss(outputs, clean_images)
                total_val_loss += loss.item() * noisy_images.size(0)

                # Calculate MSE for the batch
                mse_values = F.mse_loss(outputs, clean_images, reduction='none')
                mse_list.extend(mse_values.view(-1).cpu().numpy())

                # Calculate SSIM for the batch
                ssim_values = calculate_ssim_batch(clean_images, outputs)
                ssim_list.extend(ssim_values.view(-1).cpu().numpy().tolist())

                # ssim_list.extend()

        avg_val_loss = total_val_loss / len(val_loader.dataset)
        
        # Calculate average MSE and SSIM
        avg_mse = sum(mse_list) / len(mse_list) if mse_list else 0
        avg_ssim = np.nanmean(ssim_list)  # Use nanmean to ignore None values

        # Store validation metrics
        training_data['val_loss'].append(avg_val_loss)
        training_data['mse'].append(avg_mse)
        training_data['ssim'].append(avg_ssim)

        print(f'Epoch [{epoch+1}/{num_epochs}], '
              f'Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, '
              f'MSE: {avg_mse:.4f}, SSIM: {avg_ssim:.4f}')

    end_time = time.time()
    training_time = end_time - start_time
    training_data['training_time'] = training_time

    return training_data


In [34]:
import json
def save_model_and_results(model, model_name, training_data):
    torch.save(model.state_dict(), f'{model_name}_model.pth')
    
    # Convert training data to a JSON-serializable format
    serializable_data = {}
    for key, value in training_data.items():
        serializable_data[key] = value

    # Save the training data as JSON
    with open(f'{model_name}_training_data.json', 'w') as json_file:
        json.dump(serializable_data, json_file)


In [35]:
import copy
from torch.optim.lr_scheduler import CosineAnnealingLR


model = DenoisingAutoencoder().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = CosineAnnealingLR(optimizer, T_max=10, eta_min=0.0001)

training_data = train_and_evaluate_autoencoder(model, train_loader, val_loader, optimizer, scheduler, num_epochs=600)
save_model_and_results(model, 'denoising_autoencoder', training_data)



Validating Epoch 1/600: 100%|██████████| 1/1 [00:00<00:00,  1.62it/s]


Epoch [1/600], Train Loss: 0.2181, Val Loss: 0.1873, MSE: 0.1873, SSIM: 0.0932


Validating Epoch 2/600: 100%|██████████| 1/1 [00:00<00:00,  1.86it/s]


Epoch [2/600], Train Loss: 0.1504, Val Loss: 0.1559, MSE: 0.1559, SSIM: 0.1394


Validating Epoch 3/600: 100%|██████████| 1/1 [00:00<00:00,  1.83it/s]


...

Epoch [598/600], Train Loss: 0.0644, Val Loss: 0.0516, MSE: 0.0516, SSIM: 0.6979


Validating Epoch 599/600: 100%|██████████| 1/1 [00:00<00:00,  1.33it/s]


Epoch [599/600], Train Loss: 0.0622, Val Loss: 0.0650, MSE: 0.0650, SSIM: 0.6410


Validating Epoch 600/600: 100%|██████████| 1/1 [00:00<00:00,  1.25it/s]


Epoch [600/600], Train Loss: 0.0653, Val Loss: 0.0657, MSE: 0.0657, SSIM: 0.6070


In [36]:
import os
import torch
import torchvision.utils as vutils
from torchvision import transforms
from PIL import Image

def run_inference_and_save_images(model, test_loader, device, output_dir='output_images'):
    """
    Run inference on test data and save the resulting images to a specified directory.

    Args:
        model: The trained model for inference.
        test_loader: DataLoader object containing the test dataset.
        device: The device (CPU or GPU) to run the model on.
        output_dir: Directory where the output images will be saved.
    """
    # Create output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Set model to evaluation mode
    model.eval()

    # Disable gradient calculation for inference
    with torch.no_grad():
        for batch_idx, (inputs, _) in enumerate(test_loader):
            # Move inputs to the specified device (e.g., GPU if available)
            inputs = inputs.to(device)

            # Run inference (model prediction)
            outputs = model(inputs)

            # Assuming the output is a batch of images, we process and save them
            for i, output in enumerate(outputs):
                # Convert the tensor output to PIL Image for saving
                output_image = transforms.ToPILImage()(output.cpu())

                # Construct the output file path
                output_file = os.path.join(output_dir, f'output_{batch_idx}_{i}.png')

                # Save the image
                output_image.save(output_file)

                print(f'Saved image: {output_file}')



In [37]:
# load saved model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = DenoisingAutoencoder().to(device)
model.load_state_dict(torch.load('denoising_autoencoder_model.pth'))

C:\Users\yibor\AppData\Local\Temp\ipykernel_34048\2162444770.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('denoising_autoencoder_mode

<All keys matched successfully>

In [39]:
run_inference_and_save_images(model, test_loader, device)


Saved image: output_images\output_0_0.png
Saved image: output_images\output_0_1.png
Saved image: output_images\output_0_2.png
Saved image: output_images\output_0_3.png
Saved image: output_images\output_0_4.png
Saved image: output_images\output_0_5.png
Saved image: output_images\output_0_6.png
Saved image: output_images\output_0_7.png
Saved image: output_images\output_0_8.png
Saved image: output_images\output_0_9.png
Saved image: output_images\output_0_10.png
Saved image: output_images\output_0_11.png
Saved image: output_images\output_0_12.png
Saved image: output_images\output_0_13.png
Saved image: output_images\output_0_14.png
Saved image: output_images\output_0_15.png
Saved image: output_images\output_0_16.png
Saved image: output_images\output_0_17.png
Saved image: output_images\output_0_18.png
Saved image: output_images\output_0_19.png
Saved image: output_images\output_0_20.png
Saved image: output_images\output_0_21.png
Saved image: output_images\output_0_22.png
Saved image: output_i